In [39]:
import pandas as pd
from pulp import *

# ✅ 读取数据
file_path = r"C:\Users\zhangyutang\Desktop\附件3_处理_final_modified.xlsx"
df = pd.read_excel(file_path)
df.fillna(0, inplace=True)

# ✅ 清理列名
df.columns = df.columns.str.replace("\n", "").str.strip()
df.rename(columns={"餐次": "meal", "食物名称": "food_name"}, inplace=True)

# ✅ 处理单位克重（可半份的食物克重减半）
df["unit_weight"] = df["可食部（克/份）"]
df.loc[df["是否可半份"] == 1, "unit_weight"] /= 2

# ✅ 设置每个变量的上限（默认 10，若可半份则为 20）
df["x_upper"] = 2
df.loc[df["是否可半份"] == 1, "x_upper"] = 4

# ✅ 创建整数型决策变量 x_i
df["x_var"] = [
    LpVariable(f"x_{i}", lowBound=0, upBound=df.loc[i, "x_upper"], cat="Integer")
    for i in range(len(df))
]


C:\Users\zhangyutang\AppData\Local\Temp\ipykernel_21688\4244035253.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[50.   1.  50.   1.  75.   1.5 50.   1.5 50.   1.5 50.   1.  50.  10.
  1.  75.   1.5 50.   1.5 50.   1.  50.  10.   1.  40.  75.   2.5 75.
 75.   2.5 20.  50.  25.   2.5 75.  50.   2.5 50.   5.   2.5 50.  10.
  2.5 50.  10.   2.5 40.   5.  15.   2.5 50.   7.5  2.5 10.  50.   2.5
 10.  50.   2.5 40.  40.   5.   5.  50.  15.  15.   2.5 50.  15.  15.
  2.5 50.  15.  15.   2.5 40.  15.  15.   2.5 40.   5.  15.  15.   2.5
 75.  15.  15.   2.5 25.  25.   5.   5.   7.5 25.   5.   5.   7.5 25.
 25.   5.  25.  25.   5.  25.   5.   5.   5.   5.  50.   5.   5.   5.
 50.  75.   7.5 50.   5.   5.   5.  50.   1.  75.   1.5 50.   1.5 50.
  1.  50.  10.   1.  25.  50.  50.   2.5 20.  50.  25.   2.5 75.  50.
  2.5 50.   5.   2.5 50.  10.   2.5 50.  10.   2.5 40.   5.  15.   2.5
 50.   7.5  2.5 10.  50.

In [28]:
aas_ref = {
    "异亮氨酸 (mg/100g)": 40,
    "亮氨酸 (mg/100g)": 70,
    "赖氨酸 (mg/100g)": 55,
    "含硫氨基酸(SAA)_Total (mg/100g)": 35,
    "芳香族氨基酸(AAA)_Total (mg/100g)": 60,
    "苏氨酸 (mg/100g)": 40,
    "色氨酸 (mg/100g)": 10,
    "缬氨酸 (mg/100g)": 50,
}


In [29]:
# ✅ 创建三餐 AAS 评分变量（连续变量）
z_b = LpVariable("z_b", lowBound=0)
z_l = LpVariable("z_l", lowBound=0)
z_d = LpVariable("z_d", lowBound=0)

# ✅ 初始化模型
model = LpProblem("最大化三餐AAS评分", LpMaximize)

# ✅ 设置目标函数
model += z_b + z_l + z_d, "总AAS评分最大化"


In [30]:
def add_aas_constraints_approx(model, df, z_var, meal_name, P_ref=30):
    """
    为指定餐次添加线性近似的AAS评分约束：
        z_m <= AA_j / (R_j * P_ref)
    """
    df_sub = df[df["meal"] == meal_name]

    for aa_col, ref_value in aas_ref.items():
        aa_expr = lpSum([
            row["x_var"] * row["unit_weight"] * row[aa_col] / 100
            for _, row in df_sub.iterrows()
        ])
        model += z_var <= aa_expr / (ref_value * P_ref)


In [31]:
add_aas_constraints_approx(model, df, z_b, "早餐")
add_aas_constraints_approx(model, df, z_l, "午餐")
add_aas_constraints_approx(model, df, z_d, "晚餐")


In [32]:
# ✅ 定义总能量表达式
E_total = lpSum([
    row["x_var"] * row["unit_weight"] * row["能量 (kcal/100g)"] / 100
    for _, row in df.iterrows()
])

# ✅ 添加上下限约束
model += E_total >= 2400 * 0.9, "总能量下限_90%"
model += E_total <= 2400 * 1.1, "总能量上限_110%"


In [33]:
# 蛋白质供能（4 kcal/g）
E_p = lpSum([
    row["x_var"] * row["unit_weight"] * row["蛋白质 (g/100g)"] / 100 * 4
    for _, row in df.iterrows()
])

# 脂肪供能（9 kcal/g）
E_f = lpSum([
    row["x_var"] * row["unit_weight"] * row["脂肪 (g/100g)"] / 100 * 9
    for _, row in df.iterrows()
])

# 碳水供能（4 kcal/g）
E_c = lpSum([
    row["x_var"] * row["unit_weight"] * row["碳水化合物 (g/100g)"] / 100 * 4
    for _, row in df.iterrows()
])


In [34]:
model += E_p >= 0.10 * E_total, "蛋白质供能下限"
model += E_p <= 0.15 * E_total, "蛋白质供能上限"

model += E_f >= 0.20 * E_total, "脂肪供能下限"
model += E_f <= 0.30 * E_total, "脂肪供能上限"

model += E_c >= 0.50 * E_total, "碳水供能下限"
model += E_c <= 0.65 * E_total, "碳水供能上限"


In [38]:
# 宏量营养素推荐摄入范围（单位：g）
macro_range = {
    "蛋白质 (g/100g)": (60, 90),
    "脂肪 (g/100g)": (53.3, 80),
    "碳水化合物 (g/100g)": (300, 390),
}

for col, (lower, upper) in macro_range.items():
    expr = lpSum([
        row["x_var"] * row["unit_weight"] * row[col] / 100
        for _, row in df.iterrows()
    ])
    model += expr >= 0.95 * lower, f"{col}_摄入下限_95%1"
    model += expr <= 1.05 * upper, f"{col}_摄入上限_105%1"


In [40]:
# 微量营养素推荐摄入值（单位见下）
micronutrient_limits = {
    "钙 (mg/100g)": 800,
    "铁 (mg/100g)": 12,
    "锌 (mg/100g)": 12.5,
    "总维生素A (μg/100g)": 800,
    "硫胺素 (μg/100g)": 1400,   # μg，需要转 mg
    "核黄素 (mg/100g)": 1.4,
    "维生素C (mg/100g)": 100,
}

for col, ref in micronutrient_limits.items():
    # ✅ 单位转换：VB1字段是 μg，要除以1000转成 mg
    factor = 1 / 1000 if col == "硫胺素 (μg/100g)" else 1

    expr = lpSum([
        row["x_var"] * row["unit_weight"] * row[col] / 100 * factor
        for _, row in df.iterrows()
    ])

    model += expr >= 0.95 * ref * factor, f"{col}_摄入下限_95%"
    model += expr <= 1.05 * ref * factor, f"{col}_摄入上限_105%"


In [41]:
df["y_var"] = [
    LpVariable(f"y_{i}", cat="Binary")
    for i in range(len(df))
]

for _, row in df.iterrows():
    model += row["x_var"] <= row["x_upper"] * row["y_var"], f"链接_x_y_{row.name}"

model += lpSum(df["y_var"]) >= 12, "每日食物种类不少于12"

for meal in ["早餐", "午餐", "晚餐"]:
    model += lpSum(df[df["meal"] == meal]["y_var"]) <= 7, f"{meal}_食物种类不超过7"


In [43]:
model.solve(PULP_CBC_CMD(msg=True))  # 设置 msg=True 查看详细过程
print(f"模型求解状态：{LpStatus[model.status]}")
df["x_opt"] = df["x_var"].apply(value)
df["y_opt"] = df["y_var"].apply(value)
for meal in ["早餐", "午餐", "晚餐"]:
    print(f"\n🍽 {meal} 推荐菜品：")
    df_meal = df[(df["meal"] == meal) & (df["x_opt"] > 0)]
    for _, row in df_meal.iterrows():
        grams = row["unit_weight"] * row["x_opt"]
        print(f" - {row['food_name']}：{row['x_opt']} 份（约 {grams:.1f} 克）")
print("\n🥩 三餐 AAS评分：")
print(f" - 早餐 AAS：{value(z_b):.4f}")
print(f" - 午餐 AAS：{value(z_l):.4f}")
print(f" - 晚餐 AAS：{value(z_d):.4f}")
print(f" - 总AAS得分：{value(z_b + z_l + z_d):.4f}")
df["E_kcal"] = df["x_opt"] * df["unit_weight"] * df["能量 (kcal/100g)"] / 100
df["P_g"] = df["x_opt"] * df["unit_weight"] * df["蛋白质 (g/100g)"] / 100
df["F_g"] = df["x_opt"] * df["unit_weight"] * df["脂肪 (g/100g)"] / 100
df["C_g"] = df["x_opt"] * df["unit_weight"] * df["碳水化合物 (g/100g)"] / 100

E_total_val = df["E_kcal"].sum()
E_P = df["P_g"].sum() * 4
E_F = df["F_g"].sum() * 9
E_C = df["C_g"].sum() * 4

print(f"\n🔥 总能量摄入：{E_total_val:.1f} kcal")
print(f" - 蛋白质占比：{E_P / E_total_val:.2%}")
print(f" - 脂肪占比：{E_F / E_total_val:.2%}")
print(f" - 碳水占比：{E_C / E_total_val:.2%}")
def summarize_nutrients(df):
    nutrients = {
        "能量 (kcal)": "能量 (kcal/100g)",
        "蛋白质 (g)": "蛋白质 (g/100g)",
        "脂肪 (g)": "脂肪 (g/100g)",
        "碳水化合物 (g)": "碳水化合物 (g/100g)",
        "钙 (mg)": "钙 (mg/100g)",
        "铁 (mg)": "铁 (mg/100g)",
        "锌 (mg)": "锌 (mg/100g)",
        "维生素A (μg)": "总维生素A (μg/100g)",
        "维生素B1 (mg)": "硫胺素 (μg/100g)",  # 需换单位
        "维生素B2 (mg)": "核黄素 (mg/100g)",
        "维生素C (mg)": "维生素C (mg/100g)"
    }

    print("\n📊 营养素摄入总量汇总：")
    for name, col in nutrients.items():
        if col not in df.columns:
            continue
        factor = 1 / 1000 if "B1" in name else 1
        total = (df["x_opt"] * df["unit_weight"] * df[col] / 100 * factor).sum()
        print(f" - {name}: {total:.2f}")


PulpSolverError: Pulp: Error while executing E:\anaconda\envs\py310\lib\site-packages\pulp\apis\../solverdir/cbc/win/i64/cbc.exe